### Random Agents
The first test is a match between two agents that play 200 random moves each if the game does not end earlier.

In [ ]:
from chess_functions import chessboard

chessboard_EA = chessboard()
for i in range(400):
    game_finished = chessboard_EA.random_legal_move()
    if game_finished:
        break
    
chessboard_EA.board

### Stockfish Against a Montecarlo Search Tree Algorithm
In the cell below we have a code that simulates a match between the Stockfish Engine and a simple Montecarlo search tree that does not implement any type of intelligence. 

In [ ]:
import chess
import chess.pgn
from monte_carlo_search_tree import node, MCTS
from chess_functions import chessboard, stockfish_eng

# chessboard() is a class with some useful functions related to the chessboard. For example with chessboard.board 
# we can identify the board object.
chessboard_EA = chessboard()
# Initialization of the stockfish engine later used to pick a move.
engine = stockfish_eng()
# Initialization of the class that contains the monte-carlo search tree
mcts = MCTS()

white = 1
moves = 0
# PGN (Portable Game Notation) is an easy-to-read format which records both the moves of the game 
# (in standard algebraic notation) and any related data such as the names of the players, the winner/loser, 
# and even the date the game was played.
pgn = []
# To export your game with all headers, comments and variations, you can do it like this:
game = chess.pgn.Game()

i = 0
while((not chessboard_EA.board.is_game_over())):
    all_moves = [chessboard_EA.board.san(i) for i in list(chessboard_EA.board.legal_moves)]
    root = node()
    root.state = chessboard_EA.board
    
    if white:
        result = engine.play_best_move(chessboard_EA.board)
        print("stockfish (white) plays: ", result)
    else:
        result = mcts.mcts_pred(root, chessboard_EA.board.is_game_over(), white)
        print("EA (black) plays: ", result)    
    
    chessboard_EA.board.push_san(result)
    
    pgn.append(result)
    
    # the operator ^= Performs Bitwise OR on operands and assign value to left operand. This means that the 
    # value of white is flipped between 0 and 1 after each move.
    white ^= 1
    
    moves += 1
    
    i += 1
    if i == 5:
        break
    

Run the next cell if you want to visualize the final position on the board obtained between Stockfish and MCST.

In [ ]:
chessboard_EA.board

### Training the EA 
In the following cell we train the EA and we end-up with a model able to decide the next move to be played.

In [ ]:
import chess
from evaluation_class import evaluator
from fitness_function import *
from evolutionary_algorithm import genetic_algorithm, Agent_copy
import time

# Parameter for the training
pop_size = 18
generations = 6
mcst_epochs = 6
mcst_depth = 1


# We define an object of the class evaluator to generate the model of the NN that evaluates the board.
eval_model = evaluator()
# Instance of the fitness function, used to define the fitness of the agents.
fitness_func = fitness
# Core of the code: the evolutionary algorithm that defines the evolution generation after generation.
ga = genetic_algorithm()


start_time = time.time() # To measure the time needed to train the models.
agent, loss = ga.execute(fitness_func, eval_model.simple_eval_model(), pop_size = pop_size, generations = generations, mcst_epochs = mcst_epochs, mcst_depth = mcst_depth)

# Updating some information about the set parameters to obtain "agent". These are useful for some data analysis and 
# parameters adjustment later on.
agent.training_time = time.time() - start_time
agent.pop_size = pop_size
agent.generations = generations
agent.mcst_epochs = mcst_epochs
agent.mcst_depth = mcst_depth
agent.description = str("This model does not shown any particular skill")
agent.loss_progression = loss

# Copying the agent in a format that can be saved by the pickle module in one of the next cells
agent_copy = Agent_copy()
agent_copy.copy_agent(agent)

In [ ]:
print("fitness score of the agent: ", agent.fitness)
print("training time in hours: ", agent.training_time/60/60)

Now that the model is trained we can decide if we want to store it in an external file. This can be useful to compare multiple trained agents.

In [ ]:
import pickle

save_filename = "keras_models/trained_models.pkl"
# set the following variable to True when trained_models.pkl is empty or when you what to delete the content. To 
# delete the content insert 'yes' when prompt ask if you want to load
file_empty = False

# if the file is not empty, we load its content into a variable.
if file_empty == False:
    with open(save_filename, "rb") as f:
        trained_models = pickle.load(f)
    
# Thanks to 'load' we can decide whether to load or not the just trained model.
load = str("no")
load = input("\nUpdate the description of the model!!\nDo you want to save this model to the external file? \nyes or no")

if load == "yes":
    if file_empty:
        trained_models = []
    else:
        trained_models.append(agent_copy)

    with open(save_filename, 'wb') as f:
        pickle.dump(trained_models, f, pickle.HIGHEST_PROTOCOL)


We can now estimate the skill level reached by the EA, letting it play against Stockfish. Stockfish is an advanced chess engine that can be regulated in order to obtain up to 21 different skill levels. Notice that already at skill = 0 Stockfish is quite good, it might be considered an ELO = 800/900.

In [ ]:
from chess_functions import stockfish_eng
engine = stockfish_eng()

skill_estimation, board = engine.stockfish_vs_EA(agent, 0, True, 15, 3)

The following cells are just tests for future developments.

In [ ]:

#engine.skill_value(100, skill)
limit_strength = {'UCI_LimitStrength': 'true'}
# engine.engine._parameters.update(limit_strength)
engine.skill_value(0, 0)
engine.engine.get_parameters()

In [ ]:
import chess
board = chess.Board()

In [ ]:

result = engine.play_best_move(board)
board.push_san(result)


In [ ]:
from monte_carlo_search_tree import MCTS
mcts = MCTS()

model = agent.neural_network
                    
def evaluation(input):
    pred = model(input.reshape(1, 8, 8, 12))
    return pred 

In [ ]:
result, _ = mcts.simple_mcst(board, evaluation, epochs = mcst_epochs, depth = mcst_depth)  
board.push(result)
result

In [ ]:
board.push_san('g8f7')
#board.pop()
#board.legal_moves

In [ ]:
board


In [ ]:
from chess_functions import stockfish_eng
import chess

engine_good = stockfish_eng()
engine_bad = stockfish_eng()

engine_good.skill_value(1350, 20)
engine_bad.skill_value(1350, 10)

board = chess.Board()

while((not board.is_game_over())):   
    result = engine_bad.play_best_move(board)
    board.push_san(result)
    if board.is_game_over():
        print("engine_bad won")
    
    if (not board.is_game_over()):
        result = engine_good.play_best_move(board)
        board.push_san(result)  
        if board.is_game_over():
            print("engine_good won")
        
board

In [ ]:
engine_bad.engine.get_parameters()

### Testing EA functions

In [ ]:
from test_EA import genetic_algorithm

from evaluation_class import evaluator
from fitness_function import *
import numpy as np


# Parameter for the training
pop_size = 15
generations = 8
mcst_epochs = 5
mcst_depth = 1

# We define an object of the class evaluator to generate the model of the NN that evaluates the board.
eval_model = evaluator()
# Instance of the fitness function, used to define the fitness of the agents.
fitness_func = fitness

test_EA = genetic_algorithm()
test_agents = test_EA.execute(fitness_func, eval_model.simple_eval_model(), pop_size = pop_size, generations = generations, mcst_epochs = mcst_epochs, mcst_depth = mcst_depth)

In [ ]:
from keras.models import clone_model
def selection(agents):
            # sorting according to the fitness value of the agents, starting from the greater values
            agents = sorted(agents, key=lambda agent: agent.fitness, reverse=True)
            # printing the fitness of each agent
            print("The loss of the agents of the previous generation was: \n")
            print('\n'.join(map(str, agents)))
            # Out of the n agents we keep only 20%, in particular the first 20% of the list where the 
            # fittest are kept.
            agents = agents[:int(0.2 * len(agents))]
            
            return agents
        
def reset_fitness(agents):
            for agent in agents:
                agent.fitness = 100
                
            return agents

class Agent:
    def __init__(self, model):
        # clone_model() by keras: Clone a Functional or Sequential Model instance generating new 
        # random weights.
        self.neural_network = clone_model(model)
        self.fitness = 100
        self.game = None
        # The following variables are filled only for the final agent obtained after the training. In 
        # this way the instance Agent contains all the information to reproduce similar results.
        self.training_time = None
        self.pop_size = None
        self.generations = None
        self.mcst_epochs = None
        self.mcst_depth = None
        
        
    # The __str__ method is called when the following functions are invoked on the object and 
    # return a string: print(), str()    
    def __str__(self):
            return 'Loss: ' + str(self.fitness)

    def apply_weights(self, weights):
        self.neural_network.set_weights(weights)

def unflatten(flattened, shapes):
            # "shapes" is a list where each element is the shape of a layer of the NN.
            # "flattened" is a gene sequence of a new offspring. It has to be reordered as list where each 
            # element is contains the weights of a layer of the model.
            new_array = []
            index = 0
            for shape in shapes:
                # "size" indicates how many element of "flattened" forms a layer of weights for the NN.
                size = np.product(shape)
                new_array.append(flattened[index : index + size].reshape(shape))
                # "index" has to be update to select the elements of the next layer
                index += size
            return np.array(new_array, dtype=object)
        
def crossover(agents, network, pop_size):
            # The agents entering in this function have already been selected, i.e. they are the fittest 20%
            # of the previous generation.
            offspring = []
            
            # Given a the population size (pop_size), 80% of the new generation is obtained with crossover. 
            # Each crossover generates two new agents.
            for _ in range((pop_size - len(agents)) // 2):
                parent1 = random.choice(agents)
                parent2 = random.choice(agents)
                # Generation of two new agents, giving as input a blank NN model
                child1 = Agent(network)
                child2 = Agent(network)
                
                # get_weights(): Returns the current weights of the layer, as NumPy arrays. "shapes" is a list
                # where each element is the shape of a layer of the NN.
                shapes = [a.shape for a in parent1.neural_network.get_weights()]
                # genes1 and genes2 are a long sequence containing the weights of the NN for the parent1 and
                # parent2.
                genes1 = np.concatenate([a.flatten() for a in parent1.neural_network.get_weights()])
                genes2 = np.concatenate([a.flatten() for a in parent2.neural_network.get_weights()])
                # Picking a random point to divide the genes between parent1 and parent2
                split = random.randint(0, len(genes1)-1)

                child1_genes = np.array(genes1[0:split].tolist() + genes2[split:].tolist())
                child2_genes = np.array(genes2[0:split].tolist() + genes1[split:].tolist())
                # To use the child_genes as weights of the NN we need to structure them as list where each 
                # element is contains the weights of a layer of the model. To do so we can exploit the 
                # function unflatten.
                child1_genes = unflatten(child1_genes, shapes)
                child2_genes = unflatten(child2_genes, shapes)
                
                child1.apply_weights(list(child1_genes))
                child2.apply_weights(list(child2_genes))
                
                offspring.append(child1)
                offspring.append(child2)
            
            # The extend() method modifies the original list adding the new elements to the list.
            agents.extend(offspring)
            # agents now contains the 20% selected + the new crossover agents.
            return agents, genes1

def mutation(agents):
    i = 0
    for agent in agents:
        # A mutation happens with a 10% probability
        if random.uniform(0.0, 1.0) <= 0.1:
            print("agent ", i)
            weights = agent.neural_network.get_weights()
            shapes = [a.shape for a in weights]

            flattened = np.concatenate([a.flatten() for a in weights])
            # selection of a random index for the weights. This index is used to pick the weight to 
            # mutate.
            randint = random.randint(0, len(flattened)-1)
            print("modified gene: ", randint, flattened[randint])
            flattened[randint] = np.random.randn()
            print("new value: ", flattened[randint])
            # print(weights)

            new_array = unflatten(flattened, shapes)
            agent.apply_weights(new_array)
            
        i += 1
    return agents


test_agents[7].fitness = 190
test_agents[9].fitness = 200
test_agents[12].fitness = 130

test_agents_sel = selection(test_agents)
test_agents_sel = reset_fitness(test_agents_sel)
test_agents_cross, gene1 = crossover(test_agents_sel, eval_model.simple_eval_model(), 5)
test_agent_fin = mutation(test_agents_cross)

#print(test_agents[7].neural_network.get_weights())
# print("agent 0: ", test_agents_sel[0].neural_network.get_weights())
# print("agent 1: ", test_agents_sel[1].neural_network.get_weights())
# print("agent 2: ", test_agents_sel[2].neural_network.get_weights())

# print("gene1: ", gene1)


In [ ]:
print(len(test_agents_cross))

weights = test_agents_cross[0].neural_network.get_weights()
shapes = [a.shape for a in weights]

flattened = np.concatenate([a.flatten() for a in weights])
print("prima di mutazione: ", flattened[58])

weights1 = test_agent_fin[0].neural_network.get_weights()
shapes1 = [a.shape for a in weights1]

flattened1 = np.concatenate([a.flatten() for a in weights1])
print("dopo di mutazione: ", flattened1[58])

# print(test_agents_cross[4].neural_network.get_weights())